In [18]:
#Importamos las librerias que vamos a utilizar.
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#Definimos el DataFrame con el archivo correspondiente
df= pd.read_parquet(r'C:\Users\Alañ\Pictures\prueba\Datasets\nuevos\autos_combustion.parquet')

#Lo copiamos y hacemos las distintas transformaciones, descartando las columnas que no nos sirven
df_combustion = df.copy()
df_combustion.drop(columns=['EconomiaCombustibleCiudad','EconomiaCombustibleCarretera', 'EconomiaCombustibleCombinada','TipoTransmision',
                            'TipoMotor', 'TamanoMotor', 'CantidadCilindrosMotor','Traccion','CO2EscapeGpm', 'TipoCombustible'], inplace=True)
df_combustion.replace(to_replace=pd.NA, value=0, inplace=True)
df_combustion['CO2_Tipo'] = (df_combustion['CO2'] > df_combustion['CO2'].median()).astype(int)

#Creamos las variables dummies
df_combustion = pd.get_dummies(df_combustion, columns=['Fabricante', 'Categoria'])

#Definimos los datos dependientes e independientes
X = df_combustion.drop(['Modelo', 'Combustible', 'CO2','CO2_Tipo'], axis=1)
y = df_combustion['CO2_Tipo']

#Entrenamos el modelo
modelo = LogisticRegression(max_iter=1000)
modelo.fit(X, y)


#Funcion de recomendación de autos menos contaminantes.
def recomendacionAuto(fabricante):
    #Buscamos los autos que copartan Fabricante/Marca
    Fabricante_dummie = 'Fabricante_' + fabricante
    df_fabricante = df_combustion[df_combustion[Fabricante_dummie] == 1].copy()

    #Guardamos los datos de Modelo, COmbustible y cantidad de
    #CO2 emitido para usarlo más adelante, luego las descartamos
    modelos_autos = df_fabricante['Modelo']
    modelo_combustible = df_fabricante['Combustible']
    modelo_co2 = df_fabricante['CO2']
    df_fabricante.drop(['Modelo', 'Combustible', 'CO2','CO2_Tipo'], axis=1, inplace=True)
    
    df_fabricante = pd.get_dummies(df_fabricante)

    
    probabilidad_emision_alta = modelo.predict_proba(df_fabricante)[:, 1]
    df_fabricante['Probabilidad_Emision_Alta'] = probabilidad_emision_alta

    #Agregamos nuevamente las columnas anteriores
    df_fabricante['Modelo'] = modelos_autos
    df_fabricante['Combustible'] = modelo_combustible
    df_fabricante['CO2'] = modelo_co2

    #Filtramos los Modelos por la cantidad de emision
    df_fabricante = df_fabricante.sort_values(by='Probabilidad_Emision_Alta', ascending=True)

    #Traspasamos los datos para mostrarlos al final 
    modelos_final = pd.DataFrame(df_fabricante['Modelo'])
    modelos_final['AnoModelo'] = df_fabricante['AnoModelo']
    modelos_final['Combustible'] = df_fabricante['Combustible']
    modelos_final['CO2'] = df_fabricante['CO2']
    
    #Convertimos en lista para facilitar mostrar los datos por pantalla
    recomendaciones_modelo = modelos_final.head(5)['Modelo'].tolist()
    recomendaciones_anio = modelos_final.head(5)['AnoModelo'].tolist()
    recomendaciones_combustible = modelos_final.head(5)['Combustible'].tolist()
    
    #Mostramos por pantalla el resultado
    recomendaciones = 'Estas son modelos de autos que nosotros recomendamos para la marca ' + str(fabricante) + ' que tienen un menor impacto ambiental.\n'
    for i in range(0,len(recomendaciones_modelo)):
        recomendaciones += 'Modelo ' + str(i+1) + ': ' + str(recomendaciones_modelo[i]) + ' del año ' + str(recomendaciones_anio[i]) + ', Combustible: ' + str(recomendaciones_combustible[i]) + '\n'
    return print(recomendaciones)

In [19]:
recomendacionAuto('Audi')

Estas son modelos de autos que nosotros recomendamos para la marca Audi que tienen un menor impacto ambiental.
Modelo 1: A3 quattro del año 2022, Combustible: Hybrid Electric
Modelo 2: RS Q8 del año 2022, Combustible: Hybrid Electric
Modelo 3: S6 del año 2022, Combustible: Hybrid Electric
Modelo 4: S7 del año 2022, Combustible: Hybrid Electric
Modelo 5: S8 del año 2022, Combustible: Hybrid Electric

